In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from run import scilens_dir

In [ ]:
scilens_dir = scilens_dir + '/cache/diffusion_graph/scilens_3M/'
df = pd.read_csv(scilens_dir + 'article_details_v1.tsv.bz2', sep='\t')

In [ ]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [ ]:
df['title_vector'] = df.title.apply(lambda x: nlp(str(x)).vector)

In [ ]:
from sklearn.manifold import TSNE
tsne_model_en_2d = TSNE(perplexity=30, n_components=2, verbose=1, init='pca', n_iter=3500, random_state=32)

tsne_result = tsne_model_en_2d.fit_transform(np.stack(df['title_vector']))


In [ ]:
tsne_result.shape

In [ ]:
import seaborn as sns

plt.figure(figsize=(16,10))
sns.scatterplot(
    x = tsne_result[:,0],
    y = tsne_result[:,1],
    #hue="y",
    #palette=sns.color_palette("hls", 10),
    #data=tsne_result,
    legend="full",
    #alpha=0.3
)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from run import scilens_dir
hn_vocabulary = open(scilens_dir + 'small_files/hn_vocabulary/hn_vocabulary.txt').read().splitlines()
vectorizer = TfidfVectorizer(vocabulary=hn_vocabulary)

D = vectorizer.fit_transform(list(df.title.apply(str)))

In [ ]:
D.shape

In [ ]:
tsne_result_2 = tsne_model_en_2d.fit_transform(D.toarray())


In [ ]:
import seaborn as sns

plt.figure(figsize=(16,10))
sns.scatterplot(
    x = tsne_result_2[:,0],
    y = tsne_result_2[:,1],
    #hue="y",
    #palette=sns.color_palette("hls", 10),
    #data=tsne_result,
    legend="full",
    #alpha=0.3
)